# Build Interactive TPC-H Dashboard

Build an interactive Streamlit dashboard that visualizes TPC-H benchmark data using charts and graphs. This template showcases Streamlit's visualization capabilities within Snowflake notebooks for business intelligence and data analysis.

**What you'll accomplish:**
- Create executive dashboards with TPC-H sales data
- Build interactive charts showing regional performance
- Analyze customer order patterns and trends
- Visualize key business metrics with Streamlit

**Business Context:**
You're a data analyst creating executive dashboards to track key business metrics. Using the industry-standard TPC-H benchmark dataset (which simulates a sales/orders database), you'll build an interactive dashboard for executives to:
- Monitor sales performance across regions
- Analyze customer order patterns
- Track supplier and product metrics
- Visualize trends over time

## Step 1: Setup Environment

First, let's set up the Learning Environment and create a unique schema for our dashboard data.

In [ ]:
-- Setup Learning Environment
USE ROLE SNOWFLAKE_LEARNING_ROLE;
USE WAREHOUSE SNOWFLAKE_LEARNING_WH;
USE DATABASE SNOWFLAKE_LEARNING_DB;

-- Create unique schema for this template
SET schema_name = CONCAT(current_user(), '_streamlit_dashboard');
CREATE SCHEMA IF NOT EXISTS IDENTIFIER($schema_name);
USE SCHEMA IDENTIFIER($schema_name);

-- Clean up any existing objects from previous runs
DROP VIEW IF EXISTS REGIONAL_SALES_SUMMARY;
DROP VIEW IF EXISTS MONTHLY_SALES_TRENDS;
DROP VIEW IF EXISTS CUSTOMER_ORDER_METRICS;
DROP VIEW IF EXISTS ORDER_STATUS_DISTRIBUTION;
DROP VIEW IF EXISTS TOP_CUSTOMERS_VIEW;

SELECT 'Environment setup complete! Schema: ' || CURRENT_SCHEMA() as STATUS;

## Step 2: Explore TPC-H Sample Data

Let's explore the TPC-H benchmark dataset to understand the structure of our sales data. TPC-H contains realistic business data including orders, customers, suppliers, and geographical information.

In [ ]:
-- Examine sample orders data
SELECT 
    O_ORDERKEY,
    O_CUSTKEY, 
    O_ORDERSTATUS,
    O_TOTALPRICE,
    O_ORDERDATE,
    O_ORDERPRIORITY
FROM SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.ORDERS 
LIMIT 10;

In [ ]:
-- Examine customer data with regional information
SELECT 
    c.C_CUSTKEY,
    c.C_NAME,
    n.N_NAME as NATION,
    r.R_NAME as REGION,
    c.C_ACCTBAL
FROM SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.CUSTOMER c
JOIN SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.NATION n ON c.C_NATIONKEY = n.N_NATIONKEY  
JOIN SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.REGION r ON n.N_REGIONKEY = r.R_REGIONKEY
LIMIT 10;

## Step 3: Create Dashboard Data Views

Now let's create aggregated views that will power our dashboard. These views summarize the raw TPC-H data into business-friendly metrics.

In [ ]:
-- Regional Sales Summary: Aggregate sales performance by geographic region
CREATE OR REPLACE VIEW REGIONAL_SALES_SUMMARY AS
SELECT 
    r.R_NAME as REGION_NAME,
    COUNT(DISTINCT o.O_ORDERKEY) as TOTAL_ORDERS,
    SUM(o.O_TOTALPRICE) as TOTAL_SALES,
    AVG(o.O_TOTALPRICE) as AVG_ORDER_VALUE,
    COUNT(DISTINCT o.O_CUSTKEY) as UNIQUE_CUSTOMERS
FROM SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.ORDERS o
JOIN SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.CUSTOMER c ON o.O_CUSTKEY = c.C_CUSTKEY  
JOIN SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.NATION n ON c.C_NATIONKEY = n.N_NATIONKEY
JOIN SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.REGION r ON n.N_REGIONKEY = r.R_REGIONKEY
GROUP BY r.R_NAME
ORDER BY TOTAL_SALES DESC;

-- View the regional summary
SELECT * FROM REGIONAL_SALES_SUMMARY;

In [ ]:
-- Monthly Sales Trends: Track sales performance over time
CREATE OR REPLACE VIEW MONTHLY_SALES_TRENDS AS
SELECT 
    YEAR(O_ORDERDATE) as ORDER_YEAR,
    MONTH(O_ORDERDATE) as ORDER_MONTH,
    DATE_TRUNC('MONTH', O_ORDERDATE) as MONTH_DATE,
    COUNT(O_ORDERKEY) as ORDER_COUNT,
    SUM(O_TOTALPRICE) as MONTHLY_SALES,
    AVG(O_TOTALPRICE) as AVG_ORDER_SIZE
FROM SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.ORDERS
WHERE O_ORDERDATE >= '1995-01-01' AND O_ORDERDATE <= '1996-12-31'
GROUP BY YEAR(O_ORDERDATE), MONTH(O_ORDERDATE), DATE_TRUNC('MONTH', O_ORDERDATE)
ORDER BY ORDER_YEAR, ORDER_MONTH;

-- View recent monthly trends
SELECT * FROM MONTHLY_SALES_TRENDS LIMIT 12;

In [ ]:
-- Order Status Distribution: Analyze order completion rates
CREATE OR REPLACE VIEW ORDER_STATUS_DISTRIBUTION AS
SELECT 
    O_ORDERSTATUS as ORDER_STATUS,
    COUNT(*) as ORDER_COUNT,
    ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER(), 2) as PERCENTAGE,
    SUM(O_TOTALPRICE) as TOTAL_VALUE
FROM SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.ORDERS
GROUP BY O_ORDERSTATUS
ORDER BY ORDER_COUNT DESC;

-- View order status breakdown
SELECT * FROM ORDER_STATUS_DISTRIBUTION;

In [ ]:
-- Top Customers: Identify highest-value customers by spending
CREATE OR REPLACE VIEW TOP_CUSTOMERS_VIEW AS
SELECT 
    c.C_CUSTKEY,
    c.C_NAME as CUSTOMER_NAME,
    n.N_NAME as NATION_NAME,
    r.R_NAME as REGION_NAME,
    COUNT(o.O_ORDERKEY) as TOTAL_ORDERS,
    SUM(o.O_TOTALPRICE) as TOTAL_SPENT,
    AVG(o.O_TOTALPRICE) as AVG_ORDER_VALUE,
    MAX(o.O_ORDERDATE) as LAST_ORDER_DATE
FROM SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.CUSTOMER c
JOIN SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.ORDERS o ON c.C_CUSTKEY = o.O_CUSTKEY
JOIN SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.NATION n ON c.C_NATIONKEY = n.N_NATIONKEY
JOIN SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.REGION r ON n.N_REGIONKEY = r.R_REGIONKEY
GROUP BY c.C_CUSTKEY, c.C_NAME, n.N_NAME, r.R_NAME
ORDER BY TOTAL_SPENT DESC;

-- View top 10 customers
SELECT * FROM TOP_CUSTOMERS_VIEW LIMIT 10;

In [ ]:
-- Customer Order Metrics: Analyze orders by priority level
CREATE OR REPLACE VIEW CUSTOMER_ORDER_METRICS AS
SELECT 
    O_ORDERPRIORITY as ORDER_PRIORITY,
    COUNT(*) as ORDER_COUNT,
    SUM(O_TOTALPRICE) as TOTAL_VALUE,
    AVG(O_TOTALPRICE) as AVG_VALUE,
    MIN(O_TOTALPRICE) as MIN_VALUE,
    MAX(O_TOTALPRICE) as MAX_VALUE
FROM SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.ORDERS
GROUP BY O_ORDERPRIORITY
ORDER BY TOTAL_VALUE DESC;

-- View priority breakdown
SELECT * FROM CUSTOMER_ORDER_METRICS;

## Step 4: Build Interactive Dashboard

Now let's create an interactive Streamlit dashboard using our prepared data views. We'll build visualizations that showcase key business insights.

In [ ]:
# Import required libraries for dashboard
import streamlit as st
import plotly.express as px
import plotly.graph_objects as go
from snowflake.snowpark import Session

# Get active Snowflake session
session = get_active_session()

# Dashboard title and introduction
st.title("📊 TPC-H Business Intelligence Dashboard")
st.markdown("""
Explore key business metrics from the TPC-H benchmark dataset using interactive visualizations.
This dashboard demonstrates Snowflake's data processing capabilities with Streamlit's visualization tools.
""")
st.divider()

In [ ]:
# Key Performance Indicators
st.subheader("📈 Key Business Metrics")

# Get overall KPIs from the data
kpis = session.sql("""
    SELECT 
        COUNT(DISTINCT O_ORDERKEY) as TOTAL_ORDERS,
        COUNT(DISTINCT O_CUSTKEY) as TOTAL_CUSTOMERS,
        SUM(O_TOTALPRICE) as TOTAL_REVENUE,
        AVG(O_TOTALPRICE) as AVG_ORDER_VALUE
    FROM SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.ORDERS
""").collect()[0]

# Display metrics in columns
col1, col2, col3, col4 = st.columns(4)

with col1:
    st.metric(
        label="Total Orders",
        value=f"{kpis[0]:,}",
        help="Total number of orders in the dataset"
    )

with col2:
    st.metric(
        label="Total Customers", 
        value=f"{kpis[1]:,}",
        help="Unique customers who placed orders"
    )

with col3:
    st.metric(
        label="Total Revenue",
        value=f"${kpis[2]:,.0f}",
        help="Sum of all order values"
    )

with col4:
    st.metric(
        label="Avg Order Value",
        value=f"${kpis[3]:,.0f}",
        help="Average value per order"
    )

In [ ]:
# Regional Sales Visualization
st.subheader("🌍 Sales by Region")

# Get regional data from our view
regional_data = session.sql("""
    SELECT REGION_NAME, TOTAL_SALES, TOTAL_ORDERS, UNIQUE_CUSTOMERS
    FROM REGIONAL_SALES_SUMMARY
    ORDER BY TOTAL_SALES DESC
""").to_pandas()

# Create interactive bar chart
fig = px.bar(
    regional_data,
    x='REGION_NAME',
    y='TOTAL_SALES',
    title='Total Sales by Region',
    color='TOTAL_SALES',
    color_continuous_scale='Blues',
    hover_data=['TOTAL_ORDERS', 'UNIQUE_CUSTOMERS']
)

fig.update_layout(
    xaxis_title="Region",
    yaxis_title="Total Sales ($)",
    showlegend=False,
    height=400
)

st.plotly_chart(fig, use_container_width=True)

In [ ]:
# Monthly Sales Trends and Order Status in two columns
col1, col2 = st.columns(2)

with col1:
    st.subheader("📈 Monthly Sales Trends")
    
    # Get monthly data
    monthly_data = session.sql("""
        SELECT ORDER_YEAR, ORDER_MONTH, MONTHLY_SALES, ORDER_COUNT
        FROM MONTHLY_SALES_TRENDS
        ORDER BY ORDER_YEAR, ORDER_MONTH
    """).to_pandas()
    
    # Create date column for plotting
    monthly_data['DATE'] = monthly_data['ORDER_YEAR'].astype(str) + '-' + monthly_data['ORDER_MONTH'].astype(str).str.zfill(2)
    
    # Create line chart
    fig_line = px.line(
        monthly_data,
        x='DATE',
        y='MONTHLY_SALES',
        title='Monthly Sales Over Time',
        hover_data=['ORDER_COUNT']
    )
    
    fig_line.update_layout(height=350)
    st.plotly_chart(fig_line, use_container_width=True)

with col2:
    st.subheader("📋 Order Status Distribution")
    
    # Get order status data
    status_data = session.sql("""
        SELECT ORDER_STATUS, ORDER_COUNT, PERCENTAGE
        FROM ORDER_STATUS_DISTRIBUTION
        ORDER BY ORDER_COUNT DESC
    """).to_pandas()
    
    # Create pie chart
    fig_pie = px.pie(
        status_data,
        values='ORDER_COUNT',
        names='ORDER_STATUS',
        title='Orders by Status'
    )
    
    fig_pie.update_traces(textposition='inside', textinfo='percent+label')
    fig_pie.update_layout(height=350)
    st.plotly_chart(fig_pie, use_container_width=True)

In [ ]:
# Top Customers Table with Interactive Filter
st.subheader("🏆 Top Customers by Total Spending")

# Interactive filter for number of customers to show
num_customers = st.slider(
    "Number of top customers to display:",
    min_value=5,
    max_value=20,
    value=10,
    step=1
)

# Get top customers data
customers_data = session.sql(f"""
    SELECT 
        CUSTOMER_NAME,
        NATION_NAME,
        REGION_NAME,
        TOTAL_ORDERS,
        TOTAL_SPENT,
        AVG_ORDER_VALUE
    FROM TOP_CUSTOMERS_VIEW
    ORDER BY TOTAL_SPENT DESC
    LIMIT {num_customers}
""").to_pandas()

# Format the data for display
customers_data['TOTAL_SPENT'] = customers_data['TOTAL_SPENT'].apply(lambda x: f"${x:,.0f}")
customers_data['AVG_ORDER_VALUE'] = customers_data['AVG_ORDER_VALUE'].apply(lambda x: f"${x:,.0f}")

# Rename columns for better display
customers_data.columns = [
    'Customer Name', 'Nation', 'Region', 
    'Total Orders', 'Total Spent', 'Avg Order Value'
]

# Display as interactive table
st.dataframe(
    customers_data,
    use_container_width=True,
    hide_index=True
)

## Step 5: Cleanup and Summary

Let's clean up the objects we created and summarize what we've accomplished.

In [ ]:
-- Clean up created views
DROP VIEW IF EXISTS REGIONAL_SALES_SUMMARY;
DROP VIEW IF EXISTS MONTHLY_SALES_TRENDS;
DROP VIEW IF EXISTS CUSTOMER_ORDER_METRICS;
DROP VIEW IF EXISTS ORDER_STATUS_DISTRIBUTION;
DROP VIEW IF EXISTS TOP_CUSTOMERS_VIEW;

SELECT 'Dashboard views cleaned up successfully!' as STATUS;

## Key Takeaways

🎉 **Congratulations!** You've successfully built an interactive TPC-H business dashboard. Here's what you accomplished:

### **Streamlit Features Demonstrated:**
- **📊 Interactive Charts**: Created bar, line, and pie charts with Plotly integration
- **📈 KPI Metrics**: Displayed key business indicators with `st.metric()`
- **🎛️ Interactive Controls**: Added sliders and filters for data exploration
- **📋 Data Tables**: Showed formatted, sortable data tables
- **🎨 Layout Management**: Used columns and containers for professional layouts

### **Snowflake Capabilities Showcased:**
- **🔍 Complex Queries**: Executed multi-table joins across large datasets (1.5M+ orders)
- **⚡ High Performance**: Processed TPC-H benchmark data efficiently
- **📊 Business Intelligence**: Created meaningful aggregations and business metrics
- **🌐 Global Data**: Analyzed data across multiple regions and time periods

### **Business Insights Generated:**
- Regional sales performance comparison
- Monthly sales trends and seasonality
- Customer segmentation and top performers
- Order status and completion rates

### **Next Steps:**
- Explore additional TPC-H tables (LINEITEM, PART, SUPPLIER)
- Add more interactive filters (date ranges, product categories)
- Implement drill-down capabilities for deeper analysis
- Create additional chart types (heatmaps, scatter plots)

---

## Additional Resources

- **📚 Streamlit Documentation**: [docs.streamlit.io](https://docs.streamlit.io)
- **🔗 Snowflake Streamlit Integration**: [docs.snowflake.com](https://docs.snowflake.com/en/developer-guide/streamlit/about-streamlit)
- **📊 TPC-H Benchmark**: Industry-standard dataset for performance testing
- **🎯 Templates Hub**: [app.snowflake.com/templates](https://app.snowflake.com/templates)

**💡 Ready to build more dashboards?** This template demonstrates the power of combining Snowflake's data processing with Streamlit's visualization capabilities for enterprise-scale analytics!